In [1]:
import sys
import os
niftynet_path = '/home/tom/phd/NiftyNet-Generator-PR/NiftyNet'
sys.path.append(niftynet_path)
os.environ['CUDA_VISIBLE_DEVICES'] = ''
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from niftynet.io.image_reader import ImageReader
from niftynet.io.image_sets_partitioner import ImageSetsPartitioner
from collections import namedtuple

from niftynet.contrib.preprocessors.preprocessing import Preprocessing
from niftynet.contrib.csv_reader.sampler_csv_rows import ImageWindowDatasetCSV
from niftynet.contrib.csv_reader.sampler_resize_v2_csv import ResizeSamplerCSV as ResizeSampler
from niftynet.contrib.csv_reader.csv_reader import CSVReader

INFO:tensorflow:TensorFlow version 1.10.0
CRITICAL:tensorflow:Optional Python module SimpleITK not found, please install SimpleITK and retry if the application fails.
INFO:tensorflow:Available Image Loaders:
['nibabel', 'opencv', 'skimage', 'pillow', 'dummy'].
INFO:niftynet: Optional Python module SimpleITK not found, please install SimpleITK and retry if the application fails.
INFO:niftynet: Optional Python module SimpleITK version None not found, please install SimpleITK-None and retry if the application fails.


In [2]:
#### Some setup
NetParam = namedtuple('NetParam', 'normalise_foreground_only foreground_type multimod_foreground_type histogram_ref_file norm_type cutoff normalisation whitening')
ActionParam = namedtuple('ActionParam', 'random_flipping_axes scaling_percentage rotation_angle rotation_angle_x rotation_angle_y rotation_angle_z do_elastic_deformation num_ctrl_points deformation_sigma proportion_to_deform')
class TaskParam:
    def __init__(self, classes):
        self.image = classes
net_param = NetParam(normalise_foreground_only=False, foreground_type='threshold_plus', multimod_foreground_type = 'and', histogram_ref_file='mapping.txt', norm_type='percentile', cutoff=(0.05, 0.95), normalisation=False, whitening=True)
action_param = ActionParam(random_flipping_axes=[], scaling_percentage=[], rotation_angle=None, rotation_angle_x=None, rotation_angle_y=None, rotation_angle_z=None, do_elastic_deformation=False, num_ctrl_points=6, deformation_sigma=50, proportion_to_deform=0.9)

### 1) Create a csv of labels and show how it can be returned by the CSV Reader


In [3]:
from niftynet.utilities.download import download
download('mr_ct_regression_model_zoo_data')
labels_location = 'ct.csv'
files = [file for file in os.listdir('/home/tom/niftynet/data/mr_ct_regression/CT_zero_mean') if file.endswith('.nii.gz')]
pd.DataFrame(data=[(file.replace('.nii.gz', ''), file.replace('.nii.gz', '')) for file in files]).to_csv('label.csv', index=None, header=['subject_id', 'label'])
pd.read_csv('label.csv')

Accessing: https://github.com/NifTK/NiftyNetModelZoo
mr_ct_regression_model_zoo_data: FAIL. 
No NiftyNet example was found for mr_ct_regression_model_zoo_data.


,subject_id,label
0,RAN,RAN
1,HAL,HAL
2,MIL,MIL
3,CHA,CHA
4,GRA,GRA
5,PLA,PLA
6,NAR,NAR
7,WEB,WEB
8,PAR,PAR
9,HON,HON


In [4]:
#### Testing the CSV Reader on labels
# Make sure we accept 'Label', 'label', 'LABEL'
task_param = TaskParam(['image'])
image_data_param = {'CT': {'path_to_search': '~/niftynet/data/mr_ct_regression/CT_zero_mean', 'filename_contains': 'nii'}}
# Change csv_file -> csv_path_file, csv_data_file is a csv with data
csv_data_param = {'label': {'csv_data_file': 'label.csv', 'to_ohe': True}}
grouping_param = {'image': (['CT'])}

image_sets_partitioner = ImageSetsPartitioner().initialise(image_data_param)
image_reader = ImageReader().initialise(image_data_param, grouping_param, file_list=image_sets_partitioner.all_files)
preprocessing = Preprocessing(net_param, action_param, task_param)
normalisation_layers = preprocessing.prepare_normalisation_layers()
augmentation_layers = preprocessing.prepare_augmentation_layers()
image_reader.add_preprocessing_layers(normalisation_layers + augmentation_layers)
csv_reader = CSVReader(('label',)).initialise(csv_data_param, {'label': (['label'])}, file_list=image_sets_partitioner.all_files)
print('One sample from the csv_reader:', np.squeeze(csv_reader(idx=13)[1]['label']))
window_sizes = {'image': (100, 100, 1), 'label': (1, 1, 1)}
sampler = ResizeSampler(reader=image_reader,
                        csv_reader=csv_reader,
                        window_sizes=window_sizes,
                        num_threads=2,
                        smaller_final_batch_mode='drop',
                        batch_size=2,
                        queue_length=2)
sample = next(sampler())
print(sample['image'].shape)
print(sample['label'].shape)

INFO:niftynet: 

Number of subjects 15, input section names: ['subject_id', 'CT']
-- using all subjects (without data partitioning).

INFO:niftynet: Image reader: loading 15 subjects from sections ['CT'] as input [image]
One sample from the csv_reader: [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
INFO:niftynet: reading size of preprocessed images
(1, 100, 100, 1, 1, 1)
(1, 15, 1, 1, 1, 1)


### 2) Create a csv of features and show how it can be returned by the CSV Reader


In [5]:
from niftynet.utilities.download import download
download('mr_ct_regression_model_zoo_data')
labels_location = 'ct.csv'
files = [file.replace('.nii.gz', '') for file in os.listdir('/home/tom/niftynet/data/mr_ct_regression/CT_zero_mean') if file.endswith('.nii.gz')]

pd.DataFrame(data=[tuple([file] + list(np.random.randn(10))) for file in files]).to_csv('features.csv', index=None, header=['subject_id'] + [str(x) for x in range(10)])
pd.read_csv('features.csv')

Accessing: https://github.com/NifTK/NiftyNetModelZoo
mr_ct_regression_model_zoo_data: FAIL. 
No NiftyNet example was found for mr_ct_regression_model_zoo_data.


,subject_id,0,1,2,3,4,5,6,7,8,9
0,RAN,0.321972,0.652648,0.113185,-0.722681,-0.593930,-0.900718,-0.808985,-0.368086,-1.262447,-2.299978
1,HAL,-0.439402,0.675593,-0.657466,0.768496,-0.382948,0.474596,-0.276786,-0.392421,-0.119055,1.252547
2,MIL,1.085595,0.076346,0.345417,-0.196474,0.716523,-0.548726,0.408472,0.018024,-0.583368,-0.780949
3,CHA,-2.210596,-0.137564,0.323351,0.891198,-0.778046,0.031523,-0.463217,-0.685250,-0.172702,0.011653
4,GRA,0.829021,-0.344907,-1.552366,-2.156463,-0.772904,-0.023208,1.357851,0.409369,0.667640,-0.151443
5,PLA,0.410349,1.233560,0.108877,-0.946675,0.379490,-0.195135,-0.299669,-0.072078,0.060394,0.123195
6,NAR,0.836927,0.857035,0.874829,0.686557,-0.891095,-0.223142,0.021994,1.012295,1.178720,-1.403227
7,WEB,1.330076,-1.140574,2.268951,-1.947345,-0.734009,0.002782,-0.182076,-0.303120,1.700154,0.836683
8,PAR,-0.861265,0.076940,1.981806,-0.018995,-0.362837,-1.420218,0.347479,-0.656022,1.957113,-0.974995
9,HON,-0.133184,-0.681829,0.238010,1.288651,-0.573452,2.361199,-0.795531,-1.134068,-0.439827,-0.498937


In [6]:
task_param = TaskParam(['image'])
image_data_param = {'CT': {'path_to_search': '~/niftynet/data/mr_ct_regression/CT_zero_mean', 'filename_contains': 'nii'}}
csv_data_param = {'features': {'csv_data_file': 'features.csv', 'to_ohe': False}}
grouping_param = {'image': (['CT'])}
image_sets_partitioner = ImageSetsPartitioner().initialise(image_data_param)
image_reader = ImageReader().initialise(image_data_param, grouping_param, file_list=image_sets_partitioner.all_files)
preprocessing = Preprocessing(net_param, action_param, task_param)
normalisation_layers = preprocessing.prepare_normalisation_layers()
augmentation_layers = preprocessing.prepare_augmentation_layers()
image_reader.add_preprocessing_layers(normalisation_layers + augmentation_layers)
csv_reader = CSVReader(('features',)).initialise(csv_data_param, {'features': ['features']}, file_list=image_sets_partitioner.all_files)
print('One sample from the csv_reader:', np.squeeze(csv_reader(idx=13)[1]['features']))
window_sizes = {'image': (100, 100, 1), 'features': (1, 1, 1)}
sampler = ResizeSampler(reader=image_reader,
                        csv_reader=csv_reader,
                        window_sizes=window_sizes,
                        num_threads=2,
                        smaller_final_batch_mode='drop',
                        batch_size=2,
                        queue_length=2)
sample = next(sampler())
print(sample['image'].shape)
print(sample['features'].shape)
print(sample.keys())

INFO:niftynet: 

Number of subjects 15, input section names: ['subject_id', 'CT']
-- using all subjects (without data partitioning).

INFO:niftynet: Image reader: loading 15 subjects from sections ['CT'] as input [image]
One sample from the csv_reader: [-1.67572209 -1.73029411  1.01353478  0.35487928 -1.0441231   0.65638927
  0.54933651 -1.41610099 -1.32000264 -1.39880624]
INFO:niftynet: reading size of preprocessed images
(1, 100, 100, 1, 1, 1)
(1, 10, 1, 1, 1, 1)
dict_keys(['image_location', 'image', 'features', 'features_location'])


# Testing the CSV Reader on labels AND Features



In [9]:
# Make sure we accept 'Label', 'label', 'LABEL'
task_param = TaskParam(['image'])
image_data_param = {'CT': {'path_to_search': '~/niftynet/data/mr_ct_regression/CT_zero_mean', 'filename_contains': 'nii'}}
csv_data_param = {'label': {'csv_data_file': 'label.csv', 'to_ohe': True},
                  'features': {'csv_data_file': 'features.csv', 'to_ohe': False}}
grouping_param = {'image': (['CT'])}

image_sets_partitioner = ImageSetsPartitioner().initialise(image_data_param)
image_reader = ImageReader().initialise(image_data_param, grouping_param, file_list=image_sets_partitioner.all_files)
preprocessing = Preprocessing(net_param, action_param, task_param)
normalisation_layers = preprocessing.prepare_normalisation_layers()
augmentation_layers = preprocessing.prepare_augmentation_layers()
image_reader.add_preprocessing_layers(normalisation_layers + augmentation_layers)

csv_reader = CSVReader(('label', 'features')).initialise(csv_data_param,
                                                         {'label': (['label']), 'features': (['features'])},
                                                         file_list=image_sets_partitioner.all_files)


print('One sample from the csv_reader:', np.squeeze(csv_reader(idx=13)[1]['label']))
window_sizes = {'image': (100, 100, 1), 'label': (1, 1, 1)}
sampler = ResizeSampler(reader=image_reader,
                        csv_reader=csv_reader,
                        window_sizes=window_sizes,
                        num_threads=2,
                        smaller_final_batch_mode='drop',
                        batch_size=2,
                        queue_length=2)
sample = next(sampler())
print(sample['image'].shape)
print(sample['label'].shape)
print(sample['features'].shape)


[Layer] csv_reader_6 (input undecided)
INFO:niftynet: 

Number of subjects 15, input section names: ['subject_id', 'CT']
-- using all subjects (without data partitioning).

INFO:niftynet: Image reader: loading 15 subjects from sections ['CT'] as input [image]
One sample from the csv_reader: [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
INFO:niftynet: reading size of preprocessed images
(1, 100, 100, 1, 1, 1)
(1, 15, 1, 1, 1, 1)
(1, 10, 1, 1, 1, 1)
